In [48]:
import numpy as np
import math
from matplotlib import pyplot as plt


class Normalizer:
    """
    The Great Normalizer
    """
    mins: np.ndarray
    maxs: np.ndarray
    span: np.ndarray
    mids: np.ndarray

    def __init__(self, reference_inputs: np.ndarray):
        self.mins = np.min(reference_inputs, 0)
        self.maxs = np.max(reference_inputs, 0)
        self.span = self.maxs - self.mins
        # mids = span / 2 + mins
        # and thus:
        # mids = (maxs + mins) / 2
        self.mids = (self.maxs + self.mins) / 2

    def scale(self, set: np.ndarray):
        # k0 = (reference_inputs[0] - mids) / (span / 2)
        # and thus:
        # k0 = 2*(reference_inputs[0] - mids) / span
        return 2 * np.subtract(set, self.mids) / self.span


dataset = np.array([
    [2104, 5, 1, 45, 460],
    [1416, 3, 2, 40, 232],
    [1534, 3, 2, 30, 315],
    [1600, 4, 2, 30, 389],
    [852, 2, 1, 36, 178],
])
# inputs = dataset[:, :-1]
raw_inputs = dataset[:, :-1]
inputs = Normalizer(raw_inputs).scale(raw_inputs)
outputs = dataset[:, -1]
INITIAL_W = [0.25, 6.7, -1.5, -2.14]
INITIAL_B = 2
w = INITIAL_W
b = INITIAL_B
alpha = 1


def price(x):
    """
    input:
        `N`[1...inf) input rows
    return:
        `1D` arr of `N` predictions
    """
    return np.dot(w, x.T) + b  # w is applied to N[1...inf) rows of x. Result: 1D arr of N predictions


def prices():
    return price(inputs)


def cost():
    return np.mean((prices() - outputs) ** 2) / 2


def gradients():
    err = prices() - outputs
    gw = np.dot(err, inputs) / len(err)
    gb = np.mean(err)
    return gw, gb


def measure(a, epsilon=None):
    global w, b

    divergences = 0
    w = INITIAL_W
    b = INITIAL_B
    repeats = 0
    costs = [cost()]
    deltas = [None]
    for i in range(10000):
        repeats = i + 1
        gw, gb = gradients()
        # print((gw,gb))
        w -= a*gw
        b -= a*gb
        costs.append(cost())
        deltas.append(costs[-2] - costs[-1])
        if costs[-1] > costs[-2]:
            divergences += 1
            if divergences > 2:
                # print("WARNING: COST INCREASED 3 TIMES IN A ROW. BREAKING...")
                break
        else:
            divergences = 0
        # if i % 1000 == 0 or i == 9999:
        #     print(f"Iteration {i:4}: Cost {cost():0.2e} ",
        #           # f"dj_dw: {gw}, dj_db: {gb: 0.3e}  ",
        #           f"delta: {deltas[-1]:0.5e}",
        #           f"w: {w}, b:{b: 0.5e}")
        convergence = (deltas[-1] == 0) if (epsilon is None) else (costs[-1] < epsilon)
        if convergence:
            # print(f"FUCK DONE {len(deltas)}")
            break
    return divergences == 3, repeats, deltas, costs, a
# print(cost())
# print(gradients())
# print(prices())


In [ ]:

for i in range(10000):
    repeats = i + 1
    gw, gb = gradients()
    # print((gw,gb))
    w -= alpha*gw
    b -= alpha*gb
    costs.append(cost())
    deltas.append(costs[-2] - costs[-1])
    if costs[-1] > costs[-2]:
        divergences += 1
        if divergences > 2:
            # print("WARNING: COST INCREASED 3 TIMES IN A ROW. BREAKING...")
            break
    else:
        divergences = 0
    # if i % 1000 == 0 or i == 9999:
    #     print(f"Iteration {i:4}: Cost {cost():0.2e} ",
    #           # f"dj_dw: {gw}, dj_db: {gb: 0.3e}  ",
    #           f"delta: {deltas[-1]:0.5e}",
    #           f"w: {w}, b:{b: 0.5e}")
    convergence = (deltas[-1] == 0) if (epsilon is None) else (costs[-1] < epsilon)
    if convergence:
        # print(f"FUCK DONE {len(deltas)}")
        break